In [21]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps
import re

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU4321"

# Connect to database via CRUD Module

# Data Manipulation / Model
shelter = AnimalShelter()
df = pd.DataFrame.from_records(shelter.read({}))
# Debug: Print columns of the DataFrame
print("DataFrame columns:", df.columns)

# Attempt to drop the '_id' column if it exists
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Debug: Print the number of records and the first few rows of the DataFrame
print("Number of records:", len(df))
print("First few records:\n", df.head())

# Function to convert age strings to weeks
def age_to_weeks(age_str):
    if 'year' in age_str:
        return int(age_str.split()[0]) * 52  # 1 year = 52 weeks
    elif 'month' in age_str:
        return int(age_str.split()[0]) * 4  # 1 month = 4 weeks
    elif 'week' in age_str:
        return int(age_str.split()[0])  # already in weeks
    elif 'day' in age_str:
        return int(age_str.split()[0]) / 7  # 1 day = 1/7 weeks
    return 0

# Apply the conversion function to create a new column
df['age_in_weeks'] = df['age_upon_outcome'].apply(age_to_weeks)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Dashboard layout
app.layout = html.Div([
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
               height = 250, width = 251))], href = 'https://www.snhu.edu', target = "_blank"),
    html.Center(html.B(html.H1('Cooper David SNHU CS-340 Dashboard'))),
    html.Hr(),
    
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label':'Water Rescue', 'value': 'Water'},
            {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label':'Disaster Rescue or Individual Tracking', 'value':'Disaster'},
        ],
        value='All'
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single", #allows for a row to be selected
        selected_rows=[],
        filter_action="native", #allow for a filter
        sort_action="native", #allows sorting
        page_action="native", #enable pagination
        page_current=0, #set start page
        page_size=10, #set rows per page

    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex', 'justify-content':'center'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
   
# Update the data table based on filter selection
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    print("Filter type selected:", filter_type)
    filtered_df = df  # Start with the entire DataFrame

    if filter_type == 'Water':
        # Filter for Water Rescue conditions
        breeds = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
        filtered_df = filtered_df[
            (filtered_df['breed'].isin(breeds)) &
            (filtered_df['sex_upon_outcome'] == 'Female') &
            (filtered_df['age_in_weeks'] >= 26) &
            (filtered_df['age_in_weeks'] <= 156)
        ]
    elif filter_type == 'Mountain':
        # Filter for Mountain or Wilderness Rescue conditions
        breeds = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
        filtered_df = filtered_df[
            (filtered_df['breed'].isin(breeds)) &
            (filtered_df['sex_upon_outcome'] == 'Intact Male') &
            (filtered_df['age_in_weeks'] >= 26) &
            (filtered_df['age_in_weeks'] <= 156)
        ]
    elif filter_type in ['Disaster', 'Individual']:
        # Filter for Disaster or Individual Tracking conditions
        breeds = ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
        filtered_df = filtered_df[
            (filtered_df['breed'].isin(breeds)) &
            (filtered_df['sex_upon_outcome'] == 'Intact Male') &
            (filtered_df['age_in_weeks'] >= 20)
        ]
    elif filter_type != 'All':
        # For other types, filter by rescue_type
        filtered_df = df[df['rescue_type'] == filter_type]

    print("Number of records after filtering:", len(filtered_df))
    return filtered_df.to_dict('records')


# Change the color of a selected cell
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Call back for pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dffPie = pd.DataFrame.from_dict(viewData)

    # Check if the DataFrame is empty
    if dffPie.empty:
        return [html.Div("No data to display for the selected filter.")]

    return [
        dcc.Graph(
            figure=px.pie(dffPie, names='breed')
        )
    ]

# Call back for selecting a row and then plotting the geomarker
@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(virtualRows):
    # Austin, Texas is [30.75, -97.48]
    
    # Create the views
    if not virtualRows: # build a default view if there are no selected lines
        markerArray = (30.75, -97.48) # default marker at AAC
        toolTip = "Austin Animal Center"
        popUpHeading = "Austin Animal Center"
        popUpParagraph = "Shelter Home Location"
        
    else: # builds views based on selection
        dff = pd.DataFrame(df.iloc[virtualRows]) # convert the datatable to a dataframe
        coordLat = float(dff['location_lat'].to_string().split()[1]) # strip out the lat
        coordLong = float(dff['location_long'].to_string().split()[1]) # strip out the long
        markerArray = (coordLat, coordLong) # build the array based on selection
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']

    # return the map with a child marker
    return [dl.Map(style={'width': '700px', 'height': '450px'}, center=markerArray,
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                     ])
    ]

# Run the server
app.run_server(debug=True)


DataFrame columns: Index(['', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
       'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype',
       'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long',
       'age_upon_outcome_in_weeks'],
      dtype='object')
Number of records: 9949
First few records:
       age_upon_outcome animal_id animal_type                     breed  \
0   6          5 years   A696004         Dog  Cardigan Welsh Corgi Mix   
1   4         7 months   A733653         Cat               Siamese Mix   
2   9          3 years   A720214         Dog    Labrador Retriever Mix   
3  10         3 months   A664290         Cat    Domestic Shorthair Mix   
4  11           1 year   A721199         Dog    Dachshund Wirehair Mix   

         color date_of_birth             datetime            monthyear  \
0  Sable/White    2010-01-27  2015-01-28 10:39:00  2015-01-28T10:39:00   
1   Seal Point    2016-01-25  2016-08-27 18:11:00  2016-